Modeling the Worst: A Zombie and Ebola outbreak

MOD300: Mandatory Project 2

Collaborators:  

Abstract

Introduction

Modeling a zombie outbreak

Exercise 1

Exercise 2

Exercise 3

Exercise 4

Exercise 5

Reflection